# Введение в обработку естественного языка
## Урок 9. Языковое моделирование
### Задание
Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

In [ ]:
text = open('/content/Dostoevskiyi_F._Idiot.txt', 'rb').read().decode(encoding='cp1251')
print(text[:500])

Идиот
Федор Михайлович Достоевский


«Идиот».

Роман, в котором творческие принципы Достоевского воплощаются в полной мере, а удивительное владение сюжетом достигает подлинного расцвета. Яркая и почти болезненно талантливая история несчастного князя Мышкина, неистового Парфена Рогожина и отчаявшейся Настасьи Филипповны, много раз экранизированная и поставленная на сцене, и сейчас завораживает читателя…






Часть первая





I


В конце ноября, в оттепель, часов в девять у


In [ ]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

142 unique characters


In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
seq_length = 100

examples_per_epoch = len(text)//(seq_length+1)
examples_per_epoch

13108

In [ ]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True) #drop_remainder удаляет неполный батч (с длиной менее seq_length+1)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Идиот\r\nФедор Михайлович Достоевский\r\n\r\n\r\n«Идиот».\r\n\r\nРоман, в котором творческие принципы Достоевског'
'о воплощаются в полной мере, а удивительное владение сюжетом достигает подлинного расцвета. Яркая и п'
'очти болезненно талантливая история несчастного князя Мышкина, неистового Парфена Рогожина и отчаявше'
'йся Настасьи Филипповны, много раз экранизированная и поставленная на сцене, и сейчас завораживает чи'
'тателя…\r\n\r\n\r\n\r\n\r\n\r\n\r\nЧасть первая\r\n\r\n\r\n\r\n\r\n\r\nI\r\n\r\n\r\nВ конце ноября, в оттепель, часов в девять утра, '


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 142) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           36352     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense (Dense)               (64, None, 142)           145550    
                                                                 
Total params: 30,606,990
Trainable params: 30,606,990
No

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) #количество независимых выборок 1

sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 142)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.955711


In [ ]:
model.compile(optimizer='adam', loss=loss)

checkpoint_dir = '/content/checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=204,
    save_weights_only=True,
    )

In [ ]:
EPOCHS = 3
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/3
204/204 [==============================] - 7173s 35s/step - loss: 3.3319
Epoch 2/3
204/204 [==============================] - 7199s 35s/step - loss: 3.0237
Epoch 3/3
204/204 [==============================] - 7209s 35s/step - loss: 2.2235


In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'/content/checkpoints/ckpt_3'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            36352     
                                                                 
 lstm_4 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_1 (Dense)             (1, None, 142)            145550    
                                                                 
Total params: 30,606,990
Trainable params: 30,606,990


In [ ]:
def generate_text(model, start_string):
    # Этап оценки (генерация текста с использованием обученной модели)

    # число букв для генераци
    num_generate = 100

    # Преобразование начальной строки в числа (векторизация)
    input_eval = [char2idx[s] for s in start_string]
    #Возвращаем тензор с осью длины 1, вставленной первой в индекс
    input_eval = tf.expand_dims(input_eval, 0)

    # Пустая строка для хранения результатов
    text_generated = []

    # Низкая температура приводит к более предсказуемому тексту.
    # Более высокая температура приводит к более неожиданному тексту.
    temperature = 0.001

    # здесь batch size == 1
    # сбрасываем состояния всех слоев в модели
    model.reset_states()
    for i in range(num_generate):
        #получаем предсказания модели
        predictions = model(input_eval)
        #удаляем первую размерность в предсказании
        predictions = tf.squeeze(predictions, 0)
        # использование категориального распределения для прогнозирования символа, возвращаемого моделью
        predictions = predictions / temperature
        #извлекаем выборку из категориального распределения
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Передаем предсказанный символ в качестве следующего ввода в модель
        # вместе с предыдущим скрытым состоянием 
        # добавляем 1 первым индексом к размерности
        input_eval = tf.expand_dims([predicted_id], 0)

        #сохраняем предсказанную букву
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
text_ = generate_text(model, start_string=u"в конце ноября")
print(text_)

в конце ноябрямили и не стало и не стало и не стало и не стало и не стало и не стало и не стало и не стало и не ст
